In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from imblearn.ensemble import RUSBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC 
from sklearn.model_selection import KFold, train_test_split, cross_val_score, cross_val_predict, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from time import time
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
from typing import List
from statistics import mode

In [2]:
names = [  
#         'GradientBoostingRegressor',
        'LDA',
        'Nearest Neighbors',
#         'AdaBoostClassifier',
#         'RandomForest',
#         "Linear SVM",
        "RBF SVM",
#         "Decision Tree",
#         "sLDA",
#         "MLP",
#         'RUSBoost',
        ]

# build classifiers
classifiers = [
#             GradientBoostingRegressor(random_state=1),
            LinearDiscriminantAnalysis(),
            KNeighborsClassifier(n_neighbors=20),
#             AdaBoostClassifier(n_estimators=400, learning_rate = 0.6),
#             RandomForestClassifier(),
#             SVC(kernel="linear", C=0.025),
            SVC(gamma=2, C=1),
#             DecisionTreeClassifier(),
#             LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'),
#             MLPClassifier(random_state=1, max_iter=300),
#             RUSBoostClassifier(n_estimators = 200, random_state=1),
              ]

In [3]:
df = pd.read_csv("data/embedded_whats_cooking/embedded_train.csv")
df

,id,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,label
0,10259,0.463036,0.043366,0.148925,-0.274926,0.652883,0.115647,-0.791759,-0.007681,0.531001,...,0.354413,-0.316524,0.198050,-0.035833,0.038994,0.123171,-0.514503,-0.409309,0.312318,greek
1,25693,0.223341,0.731181,0.343865,0.254070,0.232782,-0.089698,-0.678553,0.145089,-0.042785,...,0.029557,0.033568,0.062671,-0.010367,-0.152798,-0.075444,-0.185032,-0.040249,-0.040383,southern_us
2,20130,0.411820,0.636279,0.355501,0.130934,0.442166,-0.064023,-0.844747,-0.490438,-0.478358,...,0.350878,0.293865,-0.515041,-0.317028,-0.061153,0.593661,-0.321454,-0.114020,0.256753,filipino
3,22213,-0.568426,-0.211157,0.422359,0.468056,-0.114488,-0.638892,-0.495567,0.169151,0.074504,...,-0.372135,-0.422320,0.860934,0.485809,0.015329,0.191224,0.095379,-0.069076,-0.161135,indian
4,13162,0.142577,-0.224124,0.232275,0.147007,0.154988,0.332796,-0.194241,0.187481,-0.269601,...,0.185737,0.637940,-0.538573,-0.351984,0.111684,0.232872,-0.361011,-0.579879,0.075471,indian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39769,29109,-0.300061,0.214009,0.576389,0.148372,0.060303,-0.372686,-0.369670,-0.440753,-0.819607,...,-0.715899,-0.009404,-0.057255,-0.170222,-0.227720,0.092222,0.120519,-0.384632,0.114726,irish
39770,11462,-0.176457,-0.039776,0.616104,0.014460,0.779065,0.319951,-0.651085,0.065457,-0.498473,...,0.303166,0.688057,-0.367985,-0.498265,-0.109980,0.475887,-0.001684,0.067849,0.424963,italian
39771,2238,-0.019574,0.188493,0.143884,0.178563,0.682860,-0.373238,-0.474058,0.177133,0.126927,...,-0.269083,0.186699,0.295708,-0.089674,-0.566020,0.345784,-0.369699,-0.479011,0.361121,irish
39772,41882,0.776432,0.437692,0.641111,0.406933,0.145073,0.024049,-0.334166,-0.259979,-0.174525,...,0.104134,0.544601,-1.015185,-0.371715,-0.013743,0.481623,0.141545,-0.111951,-0.463663,chinese


In [4]:
y_labels = df["label"].tolist()

print(set(y_labels))

print(len(set(y_labels)))

label_dict = {}

count = 0
for ele in set(y_labels):
    label_dict[ele] = count
    count += 1
print("="*20)
print(label_dict)

{'spanish', 'french', 'chinese', 'thai', 'moroccan', 'russian', 'brazilian', 'vietnamese', 'irish', 'italian', 'korean', 'cajun_creole', 'southern_us', 'greek', 'british', 'mexican', 'filipino', 'jamaican', 'indian', 'japanese'}
20
{'spanish': 0, 'french': 1, 'chinese': 2, 'thai': 3, 'moroccan': 4, 'russian': 5, 'brazilian': 6, 'vietnamese': 7, 'irish': 8, 'italian': 9, 'korean': 10, 'cajun_creole': 11, 'southern_us': 12, 'greek': 13, 'british': 14, 'mexican': 15, 'filipino': 16, 'jamaican': 17, 'indian': 18, 'japanese': 19}


In [5]:
encoded_labels = []
for label in y_labels:
    encoded_labels.append(label_dict[label])
print(len(encoded_labels))
print(encoded_labels[:20])

39774
[13, 12, 16, 18, 18, 17, 0, 9, 15, 9, 9, 2, 9, 15, 9, 18, 14, 9, 3, 7]


In [6]:
df["y"] = encoded_labels
df = df.drop(['label', 'id'], axis=1)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

,y,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,13,0.463036,0.043366,0.148925,-0.274926,0.652883,0.115647,-0.791759,-0.007681,0.531001,...,-0.158843,0.354413,-0.316524,0.198050,-0.035833,0.038994,0.123171,-0.514503,-0.409309,0.312318
1,12,0.223341,0.731181,0.343865,0.254070,0.232782,-0.089698,-0.678553,0.145089,-0.042785,...,0.126155,0.029557,0.033568,0.062671,-0.010367,-0.152798,-0.075444,-0.185032,-0.040249,-0.040383
2,16,0.411820,0.636279,0.355501,0.130934,0.442166,-0.064023,-0.844747,-0.490438,-0.478358,...,-0.486408,0.350878,0.293865,-0.515041,-0.317028,-0.061153,0.593661,-0.321454,-0.114020,0.256753
3,18,-0.568426,-0.211157,0.422359,0.468056,-0.114488,-0.638892,-0.495567,0.169151,0.074504,...,0.973956,-0.372135,-0.422320,0.860934,0.485809,0.015329,0.191224,0.095379,-0.069076,-0.161135
4,18,0.142577,-0.224124,0.232275,0.147007,0.154988,0.332796,-0.194241,0.187481,-0.269601,...,-0.339153,0.185737,0.637940,-0.538573,-0.351984,0.111684,0.232872,-0.361011,-0.579879,0.075471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39769,8,-0.300061,0.214009,0.576389,0.148372,0.060303,-0.372686,-0.369670,-0.440753,-0.819607,...,0.156608,-0.715899,-0.009404,-0.057255,-0.170222,-0.227720,0.092222,0.120519,-0.384632,0.114726
39770,9,-0.176457,-0.039776,0.616104,0.014460,0.779065,0.319951,-0.651085,0.065457,-0.498473,...,-0.265406,0.303166,0.688057,-0.367985,-0.498265,-0.109980,0.475887,-0.001684,0.067849,0.424963
39771,8,-0.019574,0.188493,0.143884,0.178563,0.682860,-0.373238,-0.474058,0.177133,0.126927,...,0.125620,-0.269083,0.186699,0.295708,-0.089674,-0.566020,0.345784,-0.369699,-0.479011,0.361121
39772,2,0.776432,0.437692,0.641111,0.406933,0.145073,0.024049,-0.334166,-0.259979,-0.174525,...,-0.224841,0.104134,0.544601,-1.015185,-0.371715,-0.013743,0.481623,0.141545,-0.111951,-0.463663


In [7]:
scoring = "accuracy"
data = df.to_numpy()
X = data[:, 1:]
y = data[:, 0]

score_dict = {}
time_dict = {}
models = zip(names, classifiers)

for name, model in models:
    print("The model running is: " + name)
    time_start = time()
    kfold = RepeatedStratifiedKFold()
    scores = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    time_end = time()
    score_dict[name] = scores.mean()
    time_elapsed = time_end - time_start 
    time_dict[name] = time_elapsed
    print("The average score of " + name + " is", scores.mean(), "with std of", scores.std())
    print("Time to run " + name + " is", str(time_elapsed))
    print("=" * 20)
  

The model running is: LDA
The average score of LDA is 0.7152134560018031 with std of 0.0041377045200902225
Time to run LDA is 226.42991995811462
The model running is: Nearest Neighbors
The average score of Nearest Neighbors is 0.6732212155785142 with std of 0.004963808356907633
Time to run Nearest Neighbors is 344.6660888195038
The model running is: RBF SVM


KeyboardInterrupt: 

In [ ]:
print("score_dict is", score_dict)
print("time_dict is", time_dict)

In [ ]:
classifier_name = names
accuracy_list = []
time_list = []
for name in classifier_name:
    if name == 'RandomForest' or name == "RBF SVM":
        continue
    accuracy_list.append(score_dict[name])
    time_list.append(time_dict[name])
recording = {"Accuracy": accuracy_list, "Runtime(s)":time_list}

In [ ]:
df_record = pd.DataFrame(recording, index = classifier_name)

In [ ]:
df_record.to_csv("output/classifiers_accuracy_runtime.csv")